In [1]:
from myml.nbinit import *

In [ ]:
import re
from etl.files import yaml_dump
from etl.collection.utils import move_field_to_top

2025-04-10 09:42:04 [warning  ] You're on master branch, using local env instead of STAGING=master


In [ ]:
import yaml
from collections import defaultdict
from etl.collection.explorer.migration import migrate_csv_explorer
from etl.files import yaml_dump
from etl.paths import EXPLORERS_DIR, STEP_DIR

config = migrate_csv_explorer("influenza")

In [ ]:
import yaml
from collections import defaultdict
from etl.collection.explorer.migration import migrate_csv_explorer
from etl.files import yaml_dump
from etl.paths import EXPLORERS_DIR, STEP_DIR

config = migrate_csv_explorer("influenza")
# print(yaml.dump(config))
# config = yaml.safe_load(yaml_dump(config))

definitions = defaultdict(dict)

for view in config["views"]:
    # Move to common_views
    del view['config']['timelineMinTime']

    # Create shared definitions
    for indicator in view["indicators"]['y']:
        # Move some fields into definitions
        display = indicator['display']
        for key in ('additionalInfo', 'sourceLink', 'dataPublishedBy', 'sourceName'):
            info = display[key]
            info = info.replace('\\n', '\n')

            h = "def_" + str(abs(hash(display[key])))

            definitions[key][h] = info
            display[key] = '*' + h

definitions['common_views'] = [
    {
        "config": {
            "timelineMinTime": "-4043",
        },
    }
]

config["definitions"] = definitions

config = move_field_to_top(config, "definitions")

In [19]:
# extract display for indicators

tables = defaultdict(dict)

for view in config['views']:
    for ind in view['indicators']['y']:
        table_name, col = ind['catalogPath'].split('#')
        if 'variables' not in tables[table_name]:
            tables[table_name]['variables'] = {}
        variable_entry = {
            "title": col,
            'unit': ind['display'].pop('unit')
        }
        variable_entry["display"] = {
            "name": ind['display'].pop('name')
        }
        if 'shortUnit' in ind['display']:
            variable_entry['short_unit'] = ind['display'].pop('shortUnit')
        tables[table_name]['variables'][col] = variable_entry

In [20]:
# dump explorer config
path_new = STEP_DIR / "export/explorers/who/latest/influenza.config.yml"
with open(path_new, "w") as f:
    f.write(dump_yaml_with_anchors(config))

# dump metadata for the grapher step
# path_new = STEP_DIR / "data/grapher/who/latest/flu.meta.yml"
# with open(path_new, "w") as f:
#     f.write(yaml_dump({"tables": tables}))